Imports and setup

In [ ]:
# import cv2
# import mediapipe as mp
# import csv
# import os

Create Data Folder & CSV

In [2]:
DATA_PATH = "../data"
os.makedirs(DATA_PATH, exist_ok=True)
CSV_FILE = os.path.join(DATA_PATH, "gestures.csv")

# If file doesn't exist, create header
if not os.path.isfile(CSV_FILE):
    with open(CSV_FILE, "w", newline="") as f:
        writer = csv.writer(f)
        header = ["label"] + [f"{axis}{i}" for i in range(21) for axis in ["x","y","z"]]
        writer.writerow(header)

Define Gestures label

In [3]:
GESTURES = {
    "1": "fist",
    "2": "open",
    "3": "peace",
}

Hand Landmark Extraction

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_hands.Hands(max_num_hands=1,
                    min_detection_confidence=0.7,
                    min_tracking_confidence=0.7) as hands:

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)

        if result.multi_hand_landmarks:
            for landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Hand Tracking", frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()

Capture and save data

In [ ]:
coords = [[lm.x, lm.y, lm.z] for lm in landmarks.landmark]
flat_coords = sum(coords, [])  # flatten list

with open(CSV_FILE, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([current_label] + flat_coords)